In [1]:
import pandas as pd
import seaborn as sns
#https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-county-detail.html
#https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/cc-est2020-alldata.pdf

In [2]:
data = pd.read_csv('CC-EST2020-ALLDATA-48.csv')
data.drop(columns=['SUMLEV','STATE','COUNTY'], inplace=True)
data.rename(columns={'CTYNAME':'COUNTYNAME'}, inplace=True)
select_columns = ['STNAME', 'COUNTYNAME', 'YEAR', 'AGEGRP', 
                  #totals 
                  'TOT_POP', 'TOT_MALE', 'TOT_FEMALE',
                  'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE',
                  'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 
                  #not hispanic
                  'NH_MALE', 'NH_FEMALE', 'NHWA_MALE', 'NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE',
                  'NHIA_MALE','NHIA_FEMALE','NHAA_MALE','NHAA_FEMALE','NHNA_MALE', 'NHNA_FEMALE',
                  #hispanic 
                  'H_MALE', 'H_FEMALE','HWA_MALE','HWA_FEMALE','HBA_MALE','HBA_FEMALE','HIA_MALE',
                  'HIA_FEMALE','HAA_MALE','HAA_FEMALE','HNA_MALE','HNA_FEMALE']
finaldata = data[select_columns].copy()

hmap = {1:2010, 2:2010, 3:2010, 4:2011, 5:2012, 6:2013, 7:2014, 8:2015, 9:2016, 10:2017,
        11:2018, 12:2019, 13:2020}

finaldata['YEAR'] = finaldata['YEAR'].map(hmap)
finaldata
# or 
# for index in data.index:
#     old_value = data.loc[index, 'YEAR']
#     new_value = hmap[old_value]
#     data.loc[index, 'YEAR'] = new_value

,STNAME,COUNTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,...,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE
0,Texas,Anderson County,2010,0,58458,35521,22937,25452,18774,9172,...,5701,2669,276,106,112,56,106,18,31,24
1,Texas,Anderson County,2010,1,3135,1566,1569,1235,1173,225,...,375,303,13,20,9,3,6,2,3,1
2,Texas,Anderson County,2010,2,3258,1729,1529,1362,1200,279,...,367,315,12,11,8,4,0,3,8,4
3,Texas,Anderson County,2010,3,3156,1573,1583,1256,1254,256,...,275,309,14,15,5,7,1,2,1,3
4,Texas,Anderson County,2010,4,3107,1676,1431,1303,1136,323,...,323,255,7,10,9,10,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62733,Texas,Zavala County,2020,14,535,266,269,250,260,4,...,233,245,1,0,5,2,1,0,1,1
62734,Texas,Zavala County,2020,15,477,196,281,188,273,3,...,175,254,3,1,3,1,0,1,0,1
62735,Texas,Zavala County,2020,16,312,135,177,135,172,0,...,117,158,0,1,0,1,0,0,0,0
62736,Texas,Zavala County,2020,17,208,98,110,96,110,1,...,84,96,0,0,0,0,0,0,0,0


In [3]:
#select columns 
totals_data = finaldata[['STNAME', 'COUNTYNAME', 'YEAR', 'AGEGRP', 'TOT_POP' ,'TOT_MALE', 'TOT_FEMALE',
                  'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE',
                  'AA_FEMALE', 'NA_MALE', 'NA_FEMALE']].copy()
#include all age groups 
totals_data = totals_data[totals_data['AGEGRP'] == 0].copy()

#create new columns that hold totals for each race/ethnicity group 
prefixes = totals_data.columns.str.split('_').str[0]
unique_prefixes = prefixes.unique()
for prefix in unique_prefixes:
    columns_to_combine = [col for col in totals_data.columns if col.startswith(prefix + '_')]
    
    totals_data[f'{prefix}_TOTAL'] = totals_data[columns_to_combine].sum(axis=1)

#keep the valid totals rows, not the weird ones like TOT_TOTAL, etc. 
totals_only = totals_data.loc[:, totals_data.columns.str.contains('TOTAL')].drop(columns= ['STNAME_TOTAL', 'COUNTYNAME_TOTAL', 'YEAR_TOTAL',
                                                                                  'AGEGRP_TOTAL','TOT_TOTAL'])
#filter some more for good columns 
totals_data = totals_data[['STNAME', 'COUNTYNAME', 'YEAR', 'TOT_POP', 'WA_TOTAL', 'BA_TOTAL',
                           'IA_TOTAL', 'AA_TOTAL', 'NA_TOTAL']]

#drop the state name row, and groupby county-name, and find averages for totals in each race category 
totals_data.drop(columns='STNAME', inplace=True)
grouped_data = totals_data.groupby('COUNTYNAME').mean(numeric_only=True)

#Create percentage rows to find percentage of racial groups in each county 
total_columns = [col for col in grouped_data.columns if 'TOTAL' in col]
for column in total_columns:
    percent_column = column.replace('_TOTAL', '_PERCENT')
    grouped_data[percent_column] = round(grouped_data[column] / grouped_data['TOT_POP'] * 100, 2)

#filter for percentage rows 
grouped_data = grouped_data[['WA_PERCENT', 'BA_PERCENT','IA_PERCENT','AA_PERCENT','NA_PERCENT']]

grouped_data

,WA_PERCENT,BA_PERCENT,IA_PERCENT,AA_PERCENT,NA_PERCENT
COUNTYNAME,,,,,
Anderson County,75.12,21.74,0.66,0.81,0.14
Andrews County,94.62,1.86,1.44,0.74,0.02
Angelina County,81.38,15.54,0.74,1.08,0.05
Aransas County,93.39,1.67,1.14,2.02,0.09
Archer County,96.06,0.91,1.20,0.35,0.04
...,...,...,...,...,...
Wood County,91.96,5.14,0.96,0.53,0.06
Yoakum County,95.15,1.41,1.74,0.52,0.04
Young County,95.25,1.52,1.20,0.63,0.06


In [4]:
from scipy.stats import zscore

# Calculate z-scores for each minority group
z_scores = grouped_data[['BA_PERCENT', 'IA_PERCENT', 'AA_PERCENT', 'NA_PERCENT']].apply(zscore)

# Filter for counties with significantly higher representation (> 1 standard deviation above mean)
significant_counties = z_scores[(z_scores > 1).any(axis=1)]

significant_counties.reset_index(inplace=True)
counties_with_more_minorities=significant_counties['COUNTYNAME']
counties_with_more_minorities.to_csv('demographics.csv')
